In [4]:
import os
import pandas as pd
import nltk
import random
import re

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df_data = pd.DataFrame()
for country in ["da", "sv", "no"]:
    folder_path = os.path.join("app", "lang_model", "data", country)
    files = list(filter(lambda x: x.endswith(".txt"), os.listdir(folder_path)))
    sentences = []
    for filename in files:
        single_file = os.path.join(folder_path, filename)
        with open(single_file, "r") as file:
            text = file.read().replace("\n", "")
            text = re.sub(r'\d','',text)
            sentences.extend(nltk.sent_tokenize(text))

    df_country = pd.DataFrame({"data": sentences})
    df_country["label"] = country
    df_data = pd.concat([df_data, df_country])

df_data

,data,label
0,Tanzania er administrativt inddelt i regioner...,da
1,På næste niveau er landet inddelt i distrikte...,da
2,Landsdel Midtjylland-Nordjylland er i forbinde...,da
3,"Den omfatter Østjyllands Storkreds, Nordjyllan...",da
4,Ved næste folketingsvalg vil den fordeling vær...,da
...,...,...
18910,Mens noen land har lagt ansvaret på Hamas for ...,no
18911,.,no
18912,mai vedtok FNs menneskerettsråd å nedsette en ...,no
18913,Falklandsøyenes politiske system er de styrend...,no


In [6]:
df_data.reset_index(drop=True, inplace=True)
df_data["data"] = df_data["data"].str.lower()
df_data

,data,label
0,tanzania er administrativt inddelt i regioner...,da
1,på næste niveau er landet inddelt i distrikte...,da
2,landsdel midtjylland-nordjylland er i forbinde...,da
3,"den omfatter østjyllands storkreds, nordjyllan...",da
4,ved næste folketingsvalg vil den fordeling vær...,da
...,...,...
50117,mens noen land har lagt ansvaret på hamas for ...,no
50118,.,no
50119,mai vedtok fns menneskerettsråd å nedsette en ...,no
50120,falklandsøyenes politiske system er de styrend...,no


In [7]:
uniq = df_data["data"].unique()
size = len(uniq)
random.shuffle(uniq)
split = int(0.8 * size)
train = df_data[df_data["data"].isin(uniq[:split])]
test = df_data[df_data["data"].isin(uniq[split:])]

In [9]:
train.to_hdf("app/lang_model/data/train.hdf5", "data", mode="w")
test.to_hdf("app/lang_model/data/test.hdf5", "data", mode="w")

In [10]:
len(train), len(test)

(40156, 9966)

In [11]:
test.head(20)

,data,label
2,landsdel midtjylland-nordjylland er i forbinde...,da
22,retsliberalisme er en politisk ideologisk retn...,da
23,den danske moralfilosof severin christensen,da
29,attac blev grundlagt for at åbne for en mere d...,da
31,"attac mener den nyliberalistiske politik, som ...",da
35,grundloven automatisk som forretningsministerium.,da
38,normalt sidder et forretningsministerium kun i...,da
40,var den belgiske statsminister guy verhofstadt...,da
41,skattesagen var en sag om daværende partiforma...,da
48,men afsløringen vakte ingen særlig opmærksomhe...,da


In [12]:
len(test.loc[test["label"] == "da"])

3556